In [29]:
#important libraries
import pandas as pd
import numpy as np
import nltk
import re
#importing stopwords is optional, in this case it decreased accuracy
from nltk.corpus import stopwords
import itertools
import time

In [30]:
start_time = time.time()


In [31]:
data = pd.read_csv('text_emotion.csv')
data = data.iloc[:10000,:]
data.shape

(10000, 4)

In [32]:
stopset = set(stopwords.words('english'))


In [33]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()


In [ ]:
#Pre-Processing O

In [34]:

def cleaning(text):
    txt = str(text)
    txt = re.sub(r"http\S+", "", txt)
    if len(txt) == 0:
        return 'no text'
    else:
        txt = txt.split()
        index = 0
        for j in range(len(txt)):
            if txt[j][0] == '@':
                index = j
        txt = np.delete(txt, index)
        if len(txt) == 0:
            return 'no text'
        else:
            words = txt[0]
            for k in range(len(txt)-1):
                words+= " " + txt[k+1]
            txt = words
            txt = re.sub(r'[^\w]', ' ', txt)
            if len(txt) == 0:
                return 'no text'
            else:
                txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
                txt = txt.replace("'", "")
                txt = nltk.tokenize.word_tokenize(txt)
                #data.content[i] = [w for w in data.content[i] if not w in stopset]
                for j in range(len(txt)):
                    txt[j] = lem.lemmatize(txt[j], "v")
                if len(txt) == 0:
                    return 'no text'
                else:
                    return txt

# Word Dictionary

In [35]:
data['content'] = data['content'].map(lambda x: cleaning(x))
        
data = data.reset_index(drop=True)
for i in range(len(data)):
    words = data.content[i][0]
    for j in range(len(data.content[i])-1):
        words+= ' ' + data.content[i][j+1]
    data.content[i] = words

/Users/bhagyashree/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Training Of Data

In [36]:
x = int(np.round(len(data)*0.75))
train = data.iloc[:x,:].reset_index(drop = True)
test = data.iloc[x:,:].reset_index(drop = True)

In [37]:
from textblob.classifiers import NaiveBayesClassifier as NBC

In [38]:
training_corpus = []

for k in range(len(train)):
    training_corpus.append((train.content[k], train.sentiment[k]))    
test_corpus = []

for l in range(len(test)):
    test_corpus.append((test.content[l], test.sentiment[l]))

model = NBC(training_corpus)

print(model.accuracy(test_corpus))

0.314


# Testing Of Data

In [39]:
from sklearn.metrics import classification_report

predictions = []
for m in range(len(test)):
    predictions.append(model.classify(test.content[m]))
print(classification_report(test.sentiment, predictions))
    
predictions_df = pd.DataFrame({'Content':test.content, 'Emotion_predicted':predictions, 'Emotion_actual':test.sentiment})
predictions_df.to_csv('naive_emotion_recognizer.csv', index = False)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        13
     boredom       0.00      0.00      0.00        20
       empty       0.00      0.00      0.00        51
  enthusiasm       0.00      0.00      0.00        44
         fun       0.00      0.00      0.00        49
   happiness       0.00      0.00      0.00       135
        hate       0.00      0.00      0.00       118
        love       0.00      0.00      0.00        97
     neutral       0.28      0.23      0.26       443
      relief       0.00      0.00      0.00        74
     sadness       0.28      0.24      0.26       552
    surprise       0.00      0.00      0.00       134
       worry       0.33      0.71      0.45       770

   micro avg       0.31      0.31      0.31      2500
   macro avg       0.07      0.09      0.07      2500
weighted avg       0.21      0.31      0.24      2500

processing time: 4510.504441738129 seconds


# Test File

In [40]:
predictions_df

,Content,Emotion_predicted,Emotion_actual
0,sit in altanta It s rain in boston Yay for fly...,worry,neutral
1,Unfortunately no,neutral,worry
2,i m try in lot of pain though,worry,neutral
3,Bro Martin I need you to repent Pam say we be ...,worry,worry
4,woow I wan na join the maseera but on bed rest...,sadness,worry
5,I can never clean as much of my house as I wan...,worry,worry
6,young lady in the local Chinese take order be ...,worry,surprise
7,I don t even look at their fabric unless I nee...,worry,hate
8,It mean quot luv ya quot in Zulu Xhosa Probabl...,worry,neutral
9,How inconsiderate,neutral,empty
